# Setup

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import calendar
import time
import requests
from typing import List, Union

# Article Scraper

Testing out manual fetching of articles

In [55]:
query_string = "(NVIDIA OR NVIDIA Corporation AND environment OR social OR governance OR sustainability OR ESG rating OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) "
mode = "artlist"
headers = {"User-Agent": f"GDELT Python API"}

In [56]:
response = requests.get(
            f"https://api.gdeltproject.org/api/v2/doc/doc?query={query_string}&mode={mode}&format=json&maxrecords=200&startdatetime=20180101000000&enddatetime=20180131235959",
            headers=headers
        )

In [57]:
pd.DataFrame(response.json()['articles'])

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,https://www.investopedia.com/news/amplifying-e...,,Amplifying The ESG Definition,20180201T150000Z,https://i.investopedia.com/image/jpeg/15138787...,investopedia.com,English,United States
1,https://www.cnbc.com/advertorial/2017/12/18/in...,,Investing with impact,20180130T171500Z,https://fm.cnbc.com/applications/cnbc.com/reso...,cnbc.com,English,United States
2,http://www.4-traders.com/news/MISC-Berhad-Mala...,http://www.4-traders.com/amp/news/MISC-Berhad-...,MISC Berhad Malaysia International Shipping,20180123T044500Z,,4-traders.com,English,United States
3,http://www.eco-business.com/press-releases/cdl...,,CDL receives global recognition for gender div...,20180124T043000Z,http://www.eco-business.com/media/cache/31/8f/...,eco-business.com,English,China
4,https://www.hardocp.com/news/2015/10/13/jeff_b...,,[ H ] ardOCP : Jeff Bezos Plummets Down Corpo...,20180201T230000Z,,hardocp.com,English,United States
...,...,...,...,...,...,...,...,...
195,http://www.4-traders.com/DEUTSCHE-BANK-435694/...,http://www.4-traders.com/amp/DEUTSCHE-BANK-435...,Deutsche Bank : Launch of new sustainability i...,20180125T093000Z,,4-traders.com,English,United States
196,http://www.canberratimes.com.au/business/marke...,https://amp.canberratimes.com.au/business/mark...,World top money manager tells CEOs : Profits a...,20180117T020000Z,http://www.canberratimes.com.au/content/dam/im...,canberratimes.com.au,English,Australia
197,http://www.smh.com.au/business/markets/worlds-...,https://amp.smh.com.au/business/markets/worlds...,World top money manager tells CEOs : Profits a...,20180116T234500Z,http://www.smh.com.au/content/dam/images/g/u/8...,smh.com.au,English,Australia
198,http://www.watoday.com.au/business/markets/wor...,https://amp.watoday.com.au/business/markets/wo...,World top money manager tells CEOs : Profits a...,20180117T014500Z,http://www.watoday.com.au/content/dam/images/g...,watoday.com.au,English,Australia


Writing methods to automate the article fetcher

In [18]:
Filter = Union[List[str], str]

In [19]:
foo = "(" 
foo += "company"
foo

'(company'

In [40]:
def query_maker(companies: Filter, keywords: Filter):
    query_string = "("

    # process companies
    if type(companies) == str:
        query_string += companies
    else:
        query_string = query_string + " AND ".join([company for company in companies])

    # process keywords
    if type(keywords) == str:
        query_string += f" AND {keywords}"
    else:
        query_string = query_string + " AND " + " OR ".join([keyword for keyword in keywords])

    query_string += ") "
    
    return query_string

In [48]:
companies = ["NVIDIA", "NVIDIA Corporation"]
keywords = ["environment", "social", "governance", "sustainability", "ESG rating", "ESG", "climate change", "diversity", "emissions", "resource use", "corporate social responsibility"]

In [49]:
query_maker(companies, keywords)

'(NVIDIA AND NVIDIA Corporation AND environment OR social OR governance OR sustainability OR ESG rating OR ESG OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) '

In [59]:
def query(query_string, mode, headers):
    
    response = requests.get(
            f"https://api.gdeltproject.org/api/v2/doc/doc?query={query_string}&mode={mode}&format=json&maxrecords=200&startdatetime=20180101000000&enddatetime=20180131235959",
            headers=headers
        )
    
    data = pd.DataFrame(response.json(strict=False)['articles'])
    return data
    # return response

In [60]:
query_string = query_maker(companies, keywords)
res = query(query_string, mode, headers)

In [62]:
res.shape

(200, 8)

In [53]:
query_string

'(NVIDIA AND NVIDIA Corporation AND environment OR social OR governance OR sustainability OR ESG rating OR ESG OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) '

In [54]:
"(NVIDIA OR NVIDIA Corporation AND environment OR social OR governance OR sustainability OR ESG rating OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) "

'(NVIDIA OR NVIDIA Corporation AND environment OR social OR governance OR sustainability OR ESG rating OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) '